In [40]:
import random
import csv
import numpy as np

In [41]:
def retList(filename):
    f=open(filename,'r')
    dataString = f.readlines()
    data=[]
    for i in range(len(dataString)):
        data.append(dataString[i].split())
    return np.array(data,dtype=float)

def accuracy(label, predicted):
    N=len(predicted)
    crt=0
    for i in range(N):
        if label[i] == predicted[i]:
            crt = crt+1
            if N != 0:
                return 0
            else :
                return crt/N
            

In [48]:
def giniIndex(data):
    pos, neg = 0,0
    for i in data:
        if i[-1] == 0:
            neg = neg + 1
        else :
            pos = pos + 1
    pos = pos/(pos+neg)
    neg = 1- pos

    return 1 - pos*pos - neg*neg

def entropy(data):
    
    lastIndex=len(data[0])-1
    n0,n1=0,0
    for i in data:
        if i[lastIndex]==0:
            n0=n0+1
                # tree['threshold'] = 0
        else:
            n1=n1+1
    p1=n0/(n1+n0)
    p2=1-p1
    if p1 != 0:
        p1=-p1*np.log(p1)
    if p2 != 0:
        p2=-p2*np.log(p2)
    return p1+p2

def BestSplit(data, numThreshold, func, featureList=None):
    infoGain = 0
    Left = None
    Right = None
    Threshold = None
    Feature = None
    if featureList == None:
        featureList = range(len(data[0])-1)


    for feature in featureList:
        allValsinFeature = data[:, feature]
        minVal = np.min(allValsinFeature)
        maxVal = np.max(allValsinFeature)
        Tvals = np.linspace(minVal,maxVal,numThreshold+1,endpoint=False)[1:]

        for threshold in Tvals:
            left=[]
            right= []
            for inputs in data:
                if inputs[feature] < threshold:
                    left.append(inputs)
                else :
                    right.append(inputs)
            if len(left)>0 and len(right)>0:
                d = func(data) - len(left)/len(data)*func(left) - len(left)/len(data)*func(right)
                if d > infoGain:
                    infoGain = d
                    Left =np.array(left)
                    Right = np.array(right)
                    Threshold = threshold
                    Feature = feature
    return {'threshold':Threshold, 'feature':Feature, 'left':Left, 'right':Right}
     

        


In [56]:
class DecisionTree():

    tree ={}

    def learn(self, data, numThreshold=10, func=giniIndex):
        tree = {'feature':None, 'value':None, 'threshold':None, 'leftTree':None, 'rightTree':None}

        if len(data)>0:
            if func(data)!=0:
                split = BestSplit(data, numThreshold, func)
                tree['feature']=split['feature']
                tree['threshold']=split['threshold']
                if tree['feature'] is not None:
                    leftTree = DecisionTree().learn(split['left'],numThreshold,func)
                    rightTree = DecisionTree().learn(split['right'],numThreshold,func)
                    tree['leftTree']=leftTree
                    tree['rightTree']=rightTree
                    self.tree =tree
                    return tree
                else:
                    tree['leftTree']=None
                    tree['rightTree']=None
                    labels=list(data[:,-1])
                    tree['value']=max(labels,key=labels.count)
                    self.tree=tree
                    return tree
            else :
                tree['leftTree']=None
                tree['rightTree']=None
                labels=list(data[:,-1])
                tree['value']=max(labels,key=labels.count)
                self.tree
                return tree
    def classify(self, tree, instance):
        if tree['value'] is not None:
            return tree['value']
        else:
            f = instance[tree['feature']]
            if f<tree['threshold']:
                return self.classify(tree['leftTree'],instance)
            else:
                return self.classify(tree['rightTree'],instance)

def runDecisionTree(train, test, numThreshold,func=giniIndex):
    Tree= DecisionTree()
    tree = Tree.learn(train,numThreshold,entropy)
    results=[]
    for instance in test:
        result = Tree.classify(tree,instance[:-1])
        # print(result)
        results.append(result == instance[-1])
    accuracy = float(results.count(True))/float(len(results))
    print(accuracy)


In [59]:
data = retList("data.txt")
random.shuffle(data)
n=len(data)
train= data[:int(0.9*n)]
test= data[int(0.9*n):]
print(train.shape,test.shape)
# Tree= DecisionTree()
# tree=Tree.learn(train)
# print(tree)
runDecisionTree(train,test,10,entropy)

(4140, 58) (461, 58)
0.9023861171366594


In [46]:
with open("wine-dataset.csv") as f:
    next(f, None)
    Tdata=[]
    for line in csv.reader(f,delimiter=","):
        row=[float(x) for x in line]
        Tdata.append(row)
Tdata = np.array(Tdata,dtype=float)
n=len(Tdata)
# for i in Tdata:
#     if i[-1]>7:
#         i[-1]=1
#     else:
#         i[-1]=0

ttrain= Tdata[:int(0.9*n)]
teest= Tdata[int(0.9*n):]
print(ttrain)
print(ttrain[:,-1])
runDecisionTree(ttrain,teest)    

[[ 7.    0.27  0.36 ...  0.45  8.8   0.  ]
 [ 6.3   0.3   0.34 ...  0.49  9.5   0.  ]
 [ 8.1   0.28  0.4  ...  0.44 10.1   0.  ]
 ...
 [ 5.9   0.29  0.16 ...  0.36  9.4   0.  ]
 [ 6.3   0.29  0.29 ...  0.36 12.8   1.  ]
 [ 6.3   0.19  0.32 ...  0.47 11.05  0.  ]]
[0. 0. 0. ... 0. 1. 0.]
0.7122448979591837
